In [ ]:
pip install pandas

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer, pipeline
from collections import Counter  # Import the Counter class

# Load the SQuAD dataset from Hugging Face
# You can also create a custom dataset using your CSV files
# dataset = load_dataset("squad")

# Create dataframes from your train, test, and validation CSV files
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
validation_df = pd.read_csv("validation.csv")

# Load a pre-trained DistilBERT model and tokenizer for question-answering
model_name = "distilbert-base-cased-distilled-squad"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name)

# Define a function to evaluate the model
question_answering = pipeline("question-answering", model=model, tokenizer=tokenizer)

def evaluate_model(model, eval_df):
    total_f1 = 0.0
    total_em = 0.0
    total_examples = 0

    for index, row in eval_df.iterrows():
        question = row['question']
        answer = row['answer_text']

        # Use the model to answer the question
        prediction = question_answering(question=question, context="How height is David Luiz?")

        # Calculate F1 and EM scores
        f1 = squad_f1(prediction['answer'], answer)
        em = squad_em(prediction['answer'], answer)

        total_f1 += f1
        total_em += em
        total_examples += 1

    avg_f1 = total_f1 / total_examples
    avg_em = total_em / total_examples

    print(f"Average F1 Score: {avg_f1}")
    print(f"Average EM Score: {avg_em}")

def squad_f1(pred, truth):
    # Calculate F1 score
    common = Counter(pred.split()) & Counter(truth.split())
    num_common = sum(common.values())
    if num_common == 0:
        return 0
    precision = 1.0 * num_common / len(pred.split())
    recall = 1.0 * num_common / len(truth.split())
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def squad_em(pred, truth):
    # Calculate EM (Exact Match) score
    return int(pred == truth)

# Evaluate the fine-tuned model on the test and validation data
print("Testing the model on the test dataset:")
evaluate_model(model, test_df)

print("Testing the model on the validation dataset:")
evaluate_model(model, validation_df)

# Save the model for later use
model.save_pretrained("custom_qa_model")
tokenizer.save_pretrained("custom_qa_model")


Testing the model on the test dataset:
Average F1 Score: 0.0006535947712418301
Average EM Score: 0.0
Testing the model on the validation dataset:
Average F1 Score: 0.0017094017094017096
Average EM Score: 0.0


('custom_qa_model/tokenizer_config.json',
 'custom_qa_model/special_tokens_map.json',
 'custom_qa_model/vocab.txt',
 'custom_qa_model/added_tokens.json')